# exploratory_plots

#### Nav

In [ ]:
import pandas as pd
nav = pd.read_hdf('../data/interim/nav.h5', 'table')

First we subsample the nav points so that the bokeh plots don't take up as much memory.

In [ ]:
x = nav.iloc[::10].x
y = nav.iloc[::10].y
xrot = nav.iloc[::10].xrot
yrot = nav.iloc[::10].yrot

Next we plot the nav and the rotated nav, both of which are shown in meters but both of which have been shifted. The unrotated nav is shifted into an arbitrary horizontal datum, and the rotated nav is shifted so that the lower left corner of the survey area is at (0,0).

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

# create a new figure
p = figure(tools='pan,wheel_zoom,box_zoom,reset', match_aspect=True,
           output_backend="webgl", lod_threshold=100, aspect_scale=1,
           title='Nav and the Rotated Nav (Meters with Arbitrary Horizontal Datum)')

# circles for nav points
p.circle(x-x.min()-1150, y-y.min()-1503, radius=1, fill_color='blue', fill_alpha=0.8, line_color=None)
p.circle(xrot, yrot, radius=1, fill_color='green', fill_alpha=0.8, line_color=None)

# area to be used for data selection
p.line([0, 200, 200, 0, 0], [0, 0, 220, 220, 0], line_width=2, color='black')

# show the results
show(p)

#### Area selection function
This is a function that can be used to select data based on position in the rotated and shifted coordinate system.

In [ ]:
ymax = 220
ymin = 0
xmax = 200
xmin = 0

def area_select(df):
    return df.loc[(df.xrot >= xmin) & (df.xrot <= xmax) & (df.yrot >= ymin) & (df.yrot <= ymax)]

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

p = figure(tools='pan,wheel_zoom,box_zoom,reset', match_aspect=True,
           output_backend="webgl", lod_threshold=100, aspect_scale=1,
           title='Selected Nav')

# circles for nav points
p.circle(area_select(nav).xrot.values[::10], area_select(nav).yrot.values[::10],
         radius=1, fill_color='blue', fill_alpha=0.8, line_color=None)

show(p)

#### Paros

In [ ]:
paros = pd.read_hdf('../data/interim/paros.h5', 'table')
paros.head()

In [ ]:
import numpy as np
paros_min = int(np.floor(paros.pressure.min()))
paros_max = int(np.ceil(paros.pressure.max()))
hist, edges = np.histogram(paros.pressure.values, bins=np.arange(paros_min, paros_max))

In [ ]:
p = figure(tools='pan,wheel_zoom,box_zoom,reset', x_range=(2100, 2300), y_range=(0, 4e5),
          title='Histogram of Paros Pressures')
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color='blue', line_color=None)
show(p)